In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv(r"d:\rafi\power bi\telecom\Telco-Customer-Churn.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [3]:
df['TotalCharges']=df['TotalCharges'].str.strip()

In [4]:
df['TotalCharges']=pd.to_numeric(df['TotalCharges'])

In [5]:
df['TotalCharges']=df['TotalCharges'].replace(np.nan,df['TotalCharges'].mean())

In [6]:
dup_df=df.copy()
dup_df.drop(columns=("customerID"),inplace=True)

In [7]:
en_df=pd.get_dummies(dup_df,dtype="int",drop_first=True)

In [8]:
en_df.isnull().sum()

SeniorCitizen                            0
tenure                                   0
MonthlyCharges                           0
TotalCharges                             0
gender_Male                              0
Partner_Yes                              0
Dependents_Yes                           0
PhoneService_Yes                         0
MultipleLines_No phone service           0
MultipleLines_Yes                        0
InternetService_Fiber optic              0
InternetService_No                       0
OnlineSecurity_No internet service       0
OnlineSecurity_Yes                       0
OnlineBackup_No internet service         0
OnlineBackup_Yes                         0
DeviceProtection_No internet service     0
DeviceProtection_Yes                     0
TechSupport_No internet service          0
TechSupport_Yes                          0
StreamingTV_No internet service          0
StreamingTV_Yes                          0
StreamingMovies_No internet service      0
StreamingMo

In [9]:
x=en_df.drop(columns=["Churn_Yes"])

In [10]:
y=en_df["Churn_Yes"]

In [11]:
from imblearn.over_sampling import RandomOverSampler

In [12]:
ros=RandomOverSampler(sampling_strategy="all")
ro_x,ro_y=ros.fit_resample(x,y)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train,x_test,y_train,y_test=train_test_split(ro_x,ro_y,test_size=0.30,random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [16]:
# this is best
rfc=RandomForestClassifier(n_jobs=4)
rfc.fit(x_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [17]:
print(f"Train accuracy: {rfc.score(x_train,y_train):0.5f}")
print(f"Test accuracy: {rfc.score(x_test,y_test):0.5f}")
print(f"\nAccuracy Score: {accuracy_score(y_test,rfc.predict(x_test)):0.5}")
print("\nClassification Report (Test data):\n",classification_report(y_test,rfc.predict(x_test)))

Train accuracy: 0.99848
Test accuracy: 0.88663

Accuracy Score: 0.88663

Classification Report (Test data):
               precision    recall  f1-score   support

           0       0.93      0.84      0.88      1574
           1       0.85      0.94      0.89      1531

    accuracy                           0.89      3105
   macro avg       0.89      0.89      0.89      3105
weighted avg       0.89      0.89      0.89      3105



In [18]:
import joblib
joblib.dump(rfc,"Churn_model(RFC).pkl")
print("📂📂📂 Model is Saved")

📂📂📂 Model is Saved


### Model Re-training to find Churn Probability

In [19]:
# loading our model
re_train=joblib.load("Churn_model(RFC).pkl")

In [20]:
df["Churn_Probability"]=re_train.predict_proba(x)[:,1]

In [21]:
# now we predict churn 
# if churn_probability is greater than 0.5 then Churn--> Yes else No
df["Predicted_churn"]=df["Churn_Probability"].apply(lambda x: "Yes" if x>0.5 else "No")

In [22]:
df.to_csv("Churn_Prediction.csv",index=False)
print("📂📂📂 Churn_Prediction.csv is Saved")

📂📂📂 Churn_Prediction.csv is Saved


In [23]:
feature_imp=pd.DataFrame({"Features":ro_x.columns,"Values":rfc.feature_importances_})
feature_imp.to_csv("Features_importance.csv",index=False)
print("📂📂📂 Features_importance.csv is Saved")
print(feature_imp)

📂📂📂 Features_importance.csv is Saved
                                 Features    Values
0                           SeniorCitizen  0.017686
1                                  tenure  0.173981
2                          MonthlyCharges  0.152245
3                            TotalCharges  0.178238
4                             gender_Male  0.025060
5                             Partner_Yes  0.021566
6                          Dependents_Yes  0.018588
7                        PhoneService_Yes  0.004344
8          MultipleLines_No phone service  0.004151
9                       MultipleLines_Yes  0.017302
10            InternetService_Fiber optic  0.053005
11                     InternetService_No  0.006114
12     OnlineSecurity_No internet service  0.004424
13                     OnlineSecurity_Yes  0.027588
14       OnlineBackup_No internet service  0.010159
15                       OnlineBackup_Yes  0.020638
16   DeviceProtection_No internet service  0.005578
17                   Device